In [1]:
import pandas as pd
import numpy as np

import mlflow
from hyperopt import hp, STATUS_OK, fmin, Trials, tpe
from hyperopt.pyll import scope

from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('Telcom Churn')
mlflow.sklearn.autolog(True)

2023/07/01 13:08:09 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/01 13:08:09 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [3]:
def load_data(path):
    data = pd.read_csv(path)
    data.columns = data.columns.str.replace(' ', '_').str.lower()

    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    for col in categorical_col:
        data[col] = data[col].str.replace(' ', '_').str.lower()

    data = data[data['totalcharges'] != '_']
    data['totalcharges'] = data['totalcharges'].astype('float32')
    return data

In [4]:
def prepare_data(data):

    data['churn'] = (data.churn=='yes').astype(int)
    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    numerical_col = ['tenure', 'totalcharges', 'monthlycharges']

    categorical_col.remove('customerid')
    feature_cols = categorical_col + numerical_col

    train_data, test_data = train_test_split(data, test_size=0.25,
                                            random_state=0)

    train_x = train_data.drop(['churn'], axis = 1)
    test_x = test_data.drop(['churn'], axis = 1)

    train_x = train_x[feature_cols].to_dict(orient = 'records')
    test_x = test_x[feature_cols].to_dict(orient = 'records')

    train_y = train_data.pop('churn')
    test_y = test_data.pop('churn')

    out = (train_x, train_y, test_x, test_y)
    return out

In [5]:
def log_evaluation(y_true, y_pred):

    accuracy_ = accuracy_score(y_true, y_pred)
    precision_ = precision_score(y_true, y_pred)
    recall_ = recall_score(y_true, y_pred)
    f1score_ = f1_score(y_true, y_pred)

    out = {"test_accuracy_score" : accuracy_, 
    "test_precision_score" :precision_, 
    "test_recall_score" : recall_, 
    "test_f1_score" : f1score_}
    return out

In [6]:
path = './data/Telco-Customer-Churn.csv'
data = load_data(path)
train_x, train_y, test_x, \
        test_y = prepare_data(data)

# Linear Model
c_values = range(1, 100, 10)
for c_value in c_values:

    with mlflow.start_run():
    
        mlflow.set_tag('Developer', 'Godwin')

        lr_pipeline = make_pipeline(DictVectorizer(sparse= False),
                            LogisticRegression(C =c_value))

        
        lr_pipeline.fit(train_x, train_y)

        test_pred = lr_pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)
        mlflow.log_metrics(test_output_eval)

2023/07/01 13:08:14 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/07/01 13:08:17 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")
2023/07/01 13:08:21 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/07/01 13:08:21 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2023/07/01 13:08:23 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n'

In [10]:
def single_tree_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                    DecisionTreeClassifier(**params))
       
        pipeline.fit(train_x, train_y)

        test_pred = pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)   
        
        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_accuracy_score'], 'status': STATUS_OK}

def random_forest_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        
        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                    RandomForestClassifier(**params))
       
        pipeline.fit(train_x, train_y)

        test_pred = pipeline.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred) 

        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_accuracy_score'], 'status': STATUS_OK}

def single_tree():

    space = {"max_depth": hp.randint("max_depth", 1, 15),
            'min_samples_split': hp.randint("min_samples_split", 2, 15),
            'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
            }

    best_result = fmin(fn= single_tree_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result

def random_forest(): 

    space = {"n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
             'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
             'min_samples_split': hp.randint("min_samples_split", 2, 15),
             'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
             "criterion": hp.choice("criterion", ["gini", "entropy"]),
             }

    best_result = fmin(fn=random_forest_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result


In [8]:
single_tree()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/07/01 13:09:03 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:09:05 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:09:07 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  2%|▏         | 1/50 [00:04<03:41,  4.53s/trial, best loss: -0.7963594994311718]

2023/07/01 13:09:07 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:09:09 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:09:12 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  4%|▍         | 2/50 [00:09<03:44,  4.67s/trial, best loss: -0.7963594994311718]

...

2023/07/01 13:12:26 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:12:28 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:12:30 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



 98%|█████████▊| 49/50 [03:28<00:04,  4.29s/trial, best loss: -0.7992036405005688]

2023/07/01 13:12:31 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:12:33 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:12:35 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|██████████| 50/50 [03:32<00:00,  4.25s/trial, best loss: -0.7992036405005688]


{'criterion': 1,
 'max_depth': 6,
 'min_samples_leaf': 14,
 'min_samples_split': 9}

In [11]:
random_forest()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/07/01 13:15:31 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:15:39 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:15:41 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  2%|▏         | 1/50 [00:10<08:21, 10.24s/trial, best loss: -0.8122866894197952]

2023/07/01 13:15:42 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:15:47 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:15:50 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



  4%|▍         | 2/50 [00:19<07:33,  9.44s/trial, best loss: -0.8122866894197952]

...

2023/07/01 13:22:34 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:22:39 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:22:42 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



 98%|█████████▊| 49/50 [07:10<00:09,  9.37s/trial, best loss: -0.8122866894197952]

2023/07/01 13:22:42 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.

2023/07/01 13:22:46 WARNING mlflow.sklearn: Failed to infer model signature: ('Expected one of the following types:\n- pandas.DataFrame\n- pandas.Series\n- numpy.ndarray\n- dictionary of (name -> numpy.ndarray)\n- pyspark.sql.DataFrame\n', "- scipy.sparse.csr_matrix\n- scipy.sparse.csc_matrix\n- str\n- List[str]\n- List[Dict[str, Union[str, List[str]]]]\n- Dict[str, Union[str, List[str]]]\n- bytes\nbut got '<class 'list'>'")

2023/07/01 13:22:48 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.



100%|██████████| 50/50 [07:16<00:00,  8.74s/trial, best loss: -0.8122866894197952]


{'criterion': 1,
 'max_depth': 22.0,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 3}

In [12]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [15]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.test_f1_score >0.595",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_f1_score ASC"]
)

In [16]:
for run in runs:
    print(f"run id: {run.info.run_id}, F1 Score: {run.data.metrics['test_f1_score']:.4f}")

run id: 5e938454b8844a22a133f103b333f394, F1 Score: 0.5951
run id: 0d85cc0a31d64cc787c76e6a8a98f480, F1 Score: 0.5951
run id: 57966d5693dd441fb258657af59ea066, F1 Score: 0.5985
run id: 340df86639154ebd8d026e53be8efded, F1 Score: 0.6000
run id: 9f00a33d4d4d43c1969d03d106fbf4d7, F1 Score: 0.6012


In [17]:
run_id = "9f00a33d4d4d43c1969d03d106fbf4d7"

for run in runs:
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="Custormer-churn-models")

2023/07/01 13:34:01 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/01 13:34:01 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'Custormer-churn-models'.
2023/07/01 13:34:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Custormer-churn-models, version 1


Created version '1' of model 'Custormer-churn-models'.
Registered model 'Custormer-churn-models' already exists. Creating a new version of this model...
2023/07/01 13:34:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Custormer-churn-models, version 2
Created version '2' of model 'Custormer-churn-models'.
Registered model 'Custormer-churn-models' already exists. Creating a new version of this model...
2023/07/01 13:34:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Custormer-churn-models, version 3
Created version '3' of model 'Custormer-churn-models'.
Registered model 'Custormer-churn-models' already exists. Creating a new version of this model...
2023/07/01 13:34:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Custormer-churn-models, version 4
Created version 

In [19]:
model_name = "Custormer-churn-models"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 5, stage: None


In [24]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1688232842072, current_stage='Staging', description=None, last_updated_timestamp=1688233015135, name='Custormer-churn-models', run_id='0d85cc0a31d64cc787c76e6a8a98f480', run_link=None, source='/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/mlruns/1/0d85cc0a31d64cc787c76e6a8a98f480/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [25]:
from datetime import datetime

date = datetime.today().date() 

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1688232842072, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-07-01', last_updated_timestamp=1688233027543, name='Custormer-churn-models', run_id='0d85cc0a31d64cc787c76e6a8a98f480', run_link=None, source='/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/mlruns/1/0d85cc0a31d64cc787c76e6a8a98f480/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [47]:
# train = xgb.DMatrix(train_x, label=train_y)
# valid = xgb.DMatrix(test_x, label=test_y)

def xgboost_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')
        
        model = make_pipeline(DictVectorizer(sparse= False),
                              xgb.XGBClassifier(num_boost_round = 1000, **params))
        # model = xgb.train(
        #     params=params,
        #     dtrain=train,
        #     num_boost_round=1000,
        #     evals=[(valid, 'validation')],
        #     early_stopping_rounds=50
        # ) 

        model.fit(train_x, train_y)
        test_pred = model.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)  

        mlflow.log_metrics(test_output_eval)
    return {'loss': -test_output_eval['test_f1_score'], 'status': STATUS_OK}

def xgboost_dev():

    search_space = {
                    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
                    'learning_rate': hp.loguniform('learning_rate', -3, 0),
                    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
                    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
                    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
                    'objective': 'binary:logistic',
                    'seed': 42
                     }

    best_result = fmin(
                        fn= xgboost_objective,
                        space=search_space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result
mlflow.xgboost.autolog(True)

2023/07/01 14:24:51 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


In [46]:
xgboost_dev()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/07/01 14:22:51 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('dictvectorizer', DictVectorizer(sparse=False)), ('xgbclassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.12370540799044827,
     ...`

2023/07/01 14:22:51 WARNING mlflow.utils: Truncated the value of the key `xgbclassifier`. Truncated value: `XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None

  2%|▏         | 1/50 [00:09<07:32,  9.23s/trial, best loss: -0.572463768115942]

2023/07/01 14:23:00 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('dictvectorizer', DictVectorizer(sparse=False)), ('xgbclassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1871203004704139,
      ...`

2023/07/01 14:23:00 WARNING mlflow.utils: Truncated the value of the key `xgbclassifier`. Truncated value: `XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None

  4%|▍         | 2/50 [00:18<07:12,  9.02s/trial, best loss: -0.572463768115942]

...

2023/07/01 14:23:54 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('dictvectorizer', DictVectorizer(sparse=False)), ('xgbclassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1114877935663594,
      ...`

2023/07/01 14:23:54 WARNING mlflow.utils: Truncated the value of the key `xgbclassifier`. Truncated value: `XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None

 16%|█▌        | 8/50 [01:14<06:49,  9.76s/trial, best loss: -0.572463768115942]

2023/07/01 14:24:05 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('dictvectorizer', DictVectorizer(sparse=False)), ('xgbclassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.6343366483204703,
      ...`

2023/07/01 14:24:05 WARNING mlflow.utils: Truncated the value of the key `xgbclassifier`. Truncated value: `XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None

 16%|█▌        | 8/50 [01:17<06:44,  9.64s/trial, best loss: -0.572463768115942]


KeyboardInterrupt: 